In [1]:
import pandas as pd
import sqlite3 as sql

song_length_query = \
"""
SELECT Name, 
Milliseconds AS "Length in msec",
CASE
    WHEN Milliseconds < 60000 THEN "Short"
    WHEN Milliseconds > 600000 THEN "Long"
    ELSE "Moderate"
END AS "Song Length"
FROM tracks
"""

with sql.connect('data/chinook.db') as chin_con:
    song_length_df = pd.read_sql(song_length_query, chin_con)

song_length_df

,Name,Length in msec,Song Length
0,For Those About To Rock (We Salute You),343719,Moderate
1,Balls to the Wall,342562,Moderate
2,Fast As a Shark,230619,Moderate
3,Restless and Wild,252051,Moderate
4,Princess of the Dawn,375418,Moderate
...,...,...,...
3498,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,286741,Moderate
3499,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",139200,Moderate
3500,"L'orfeo, Act 3, Sinfonia (Orchestra)",66639,Moderate
3501,"Quintet for Horn, Violin, 2 Violas, and Cello ...",221331,Moderate


In [2]:
genres_query = \
"""
SELECT genres.Name, COUNT(*) AS "num"
FROM genres
LEFT JOIN tracks
    ON tracks.Genreid = genres.Genreid
GROUP BY genres.Name
ORDER BY num DESC
"""

with sql.connect('data/chinook.db') as chin_con:
    genres_df = pd.read_sql(genres_query, chin_con)

genres_df

,Name,num
0,Rock,1297
1,Latin,579
2,Metal,374
3,Alternative & Punk,332
4,Jazz,130
5,TV Shows,93
6,Blues,81
7,Classical,74
8,Drama,64
9,R&B/Soul,61


4a) Since we need to identify the genre to determine how many tracks are in each, I used Left Join on the table genres to show all the records of the genre names and its corresponding genre id to connect to the tracks table to count the number of tracks corresponding to the genre id, then identifying its genre name. The number of tracks to each genre are as follows: 
0   Rock	1297
1	Latin	579
2	Metal	374
3	Alternative & Punk	332
4	Jazz	130
5	TV Shows	93
6	Blues	81
7	Classical	74
8	Drama	64
9	R&B/Soul	61
10	Reggae	58
11	Pop	48
12	Soundtrack	43
13	Alternative	40
14	Hip Hop/Rap	35
15	Electronica/Dance	30
16	World	28
17	Heavy Metal	28
18	Sci Fi & Fantasy	26
19	Easy Listening	24
20	Comedy	17
21	Bossa Nova	15
22	Science Fiction	13
23	Rock And Roll	12
24	Opera	1

In [3]:
albums_query = \
"""
SELECT albums.Title, COUNT(*) AS "num"
FROM albums
LEFT JOIN tracks
    ON albums.Albumid = tracks.Albumid
GROUP BY albums.Albumid
ORDER BY num DESC

"""

with sql.connect('data/chinook.db') as chin_con:
    albums_df = pd.read_sql(albums_query, chin_con)

albums_df

,Title,num
0,Greatest Hits,57
1,Minha Historia,34
2,Unplugged,30
3,"Lost, Season 3",26
4,"Lost, Season 1",25
...,...,...
342,Respighi:Pines of Rome,1
343,Schubert: The Late String Quartets & String Qu...,1
344,Monteverdi: L'Orfeo,1
345,Mozart: Chamber Music,1


In [4]:
albums_query = \
"""
SELECT albums.Title, COUNT(*) AS "num"
FROM albums
LEFT JOIN tracks
    ON albums.Albumid = tracks.Albumid
GROUP BY albums.Albumid
HAVING num = 1
ORDER BY num DESC
"""

with sql.connect('data/chinook.db') as chin_con:
    albums_df = pd.read_sql(albums_query, chin_con)

albums_df

,Title,num
0,Balls to the Wall,1
1,Bark at the Moon (Remastered),1
2,Diary of a Madman (Remastered),1
3,Battlestar Galactica: The Story So Far,1
4,Un-Led-Ed,1
...,...,...
77,Respighi:Pines of Rome,1
78,Schubert: The Late String Quartets & String Qu...,1
79,Monteverdi: L'Orfeo,1
80,Mozart: Chamber Music,1


4b) The album with the largest number of tracks is "Greatest Hits". There are 82 albums with the smallest number of tracks (the smallest number of tracks for an album is 1). 

In [8]:
playlist_query = \
"""
SELECT playlists.Name, COUNT(tracks.TrackId) AS "num of tracks"
FROM playlists
JOIN playlist_track
    ON playlists.PlaylistId = playlist_track.PlaylistId
JOIN tracks
    ON tracks.TrackId = playlist_track.TrackId 
GROUP BY playlists.Name
ORDER BY "num of tracks" DESC
"""

with sql.connect('data/chinook.db') as chin_con:
    playlist_df = pd.read_sql(playlist_query, chin_con)

playlist_df

,Name,num of tracks
0,Music,6580
1,90’s Music,1477
2,TV Shows,426
3,Classical,75
4,Brazilian Music,39
5,Heavy Metal Classic,26
6,Classical 101 - The Basics,25
7,Classical 101 - Next Steps,25
8,Classical 101 - Deep Cuts,25
9,Grunge,15


4c) The playlist with the largest number of tracks is "Music"

### Which track is the most popularly sold to customers? Which is the least sold? 

In [12]:
with sql.connect('data/chinook.db') as chin_con:
    track_query = pd.read_sql("""
    SELECT tracks.Name, COUNT(invoice_items.Trackid) AS "num sold"
    FROM tracks
    JOIN invoice_items
        ON tracks.TrackId = invoice_items.TrackId 
    GROUP BY tracks.Name
    ORDER BY "num sold" DESC """, chin_con)

track_query

,Name,num sold
0,The Trooper,5
1,Untitled,4
2,The Number Of The Beast,4
3,Sure Know Something,4
4,Hallowed Be Thy Name,4
...,...,...
1883,(White Man) In Hammersmith Palais,1
1884,(Anesthesia) Pulling Teeth,1
1885,'Round Midnight,1
1886,#9 Dream,1


4di) The track that is sold the most number of times is "The Trooper", with 5 sold throughout all invoices. 

### Which artist appears the most times in all of the playlists? 

In [36]:
with sql.connect('data/chinook.db') as chin_con:
    artist_query = pd.read_sql("""
    SELECT artists.Name, COUNT(playlist_track.TrackId) AS "num of playlist appearances"
    FROM artists
    JOIN albums
        ON artists.ArtistId = albums.ArtistId
    JOIN tracks
        ON albums.AlbumId = tracks.AlbumId
    JOIN playlist_track
        ON tracks.TrackId = playlist_track.TrackId 
    GROUP BY artists.ArtistId
    ORDER BY "num of playlist appearances" DESC 
    LIMIT 1""", chin_con)

artist_query

,Name,num of playlist appearances
0,Iron Maiden,516


4dii) The artist that appears the most number of times in all the playlists combined is Iron Maiden.